In [ ]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split
from torch.utils.data import DataLoader, Dataset, Subset
from torch.utils.data import random_split, SubsetRandomSampler
from torchvision import datasets, transforms, models 
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
import torch.optim as optim 
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from PIL import Image

In [ ]:
dataset =datasets.ImageFolder(root="/kaggle/input/mepco-tropic-leaf/MepcoTropicLeaf-V1/Database",transform=None)

class_names=dataset.classes
print(class_names)
print(len(class_names))


In [ ]:
# for calculating the accuracy
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


# base class for the model
class ImageClassificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch
        out = self(images)                   # Generate prediction
        loss = F.cross_entropy(out, labels)  # Calculate loss
        acc = accuracy(out, labels)          # Calculate accuracy
        return {"val_loss": loss.detach(), "val_accuracy": acc}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x["val_loss"] for x in outputs]
        batch_accuracy = [x["val_accuracy"] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()       # Combine loss  
        epoch_accuracy = torch.stack(batch_accuracy).mean()
        return {"val_loss": epoch_loss, "val_accuracy": epoch_accuracy} # Combine accuracies
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_accuracy']))

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])


In [ ]:
data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomAffine(30),
        transforms.RandomResizedCrop((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'test':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ])
}

image_datasets = {
    'train': datasets.ImageFolder(root="/kaggle/input/mepco-tropic-leaf/MepcoTropicLeaf-V1/Database", transform=data_transforms['train']),
    'validation': datasets.ImageFolder(root="/kaggle/input/mepco-tropic-leaf/MepcoTropicLeaf-V1/Database", transform=data_transforms['validation']),
    'test': datasets.ImageFolder(root="/kaggle/input/mepco-tropic-leaf/MepcoTropicLeaf-V1/Database", transform=data_transforms['test']),
}


train_size = int(0.75 * len(image_datasets['train']))
val_size = int(0.15 * len(image_datasets['train']))
test_size = len(image_datasets['train']) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(image_datasets['train'], [train_size, val_size, test_size])

dataloaders = {
     'train':
     torch.utils.data.DataLoader(image_datasets['train'],
                                 batch_size=16,
                                 shuffle=True
                                             ),
     'validation':
     torch.utils.data.DataLoader(image_datasets['validation'],
                                 batch_size=16,
                                 shuffle=False
                                              ),
     'test':
     torch.utils.data.DataLoader(image_datasets['test'],
                                 batch_size=16,
                                 shuffle=False
                                  )
}

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = models.resnet152(pretrained=True).to(device)
    
for param in model.parameters():
    param.requires_grad = True   


In [ ]:

model.fc = nn.Sequential(
                nn.Linear(2048, 128),
                nn.LayerNorm(128),
                nn.ReLU(inplace=True),
                nn.Dropout(p=0.2),
#                 nn.Linear(512, 256),
#                 nn.LayerNorm(256),
#                 nn.ReLU(inplace=True),
#                 nn.Dropout(p=0.3),
                nn.Linear(128, 50)
                ).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(),  lr=0.01)
from torch.optim.lr_scheduler import ReduceLROnPlateau
# scheduler = ReduceLROnPlateau(optimizer, factor=0.1, patience=3)


In [ ]:
def train_model(model, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
#                   scheduler.step(loss)  # Pass the validation loss to the scheduler


                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model

In [ ]:
model_trained = train_model(model, criterion, optimizer, num_epochs=20)

In [ ]:
model_trained1 = train_model(model_trained, criterion, optimizer, num_epochs=10)

In [ ]:
model_trained2 = train_model(model_trained1, criterion, optimizer, num_epochs=20)

In [ ]:
torch.save(model_trained2.state_dict(), 'models/pytorch/weights2.h5')

In [ ]:
model = models.resnet152(pretrained=False).to(device)
model.fc = nn.Sequential(
                nn.Linear(2048, 128),
                nn.LayerNorm(128),
                nn.ReLU(inplace=True),
                nn.Dropout(p=0.2),
#                 nn.Linear(512, 256),
#                 nn.LayerNorm(256),
#                 nn.ReLU(inplace=True),
#                 nn.Dropout(p=0.3),
                nn.Linear(128, 50)
                ).to(device)
model.load_state_dict(torch.load('models/pytorch/weights2.h5'))

In [ ]:
model.eval()
y_pred = []
with torch.no_grad():
    for inputs, labels in dataloaders['test']:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_pred.extend(preds.tolist())

In [ ]:
y_true = []
with torch.no_grad():
    for inputs, labels in dataloaders['test']:
        y_true.append(labels.tolist())

In [ ]:
print(y_pred)


In [ ]:
from itertools import chain
y_true_flat = list(chain.from_iterable(y_true))
print(y_true_flat)

In [ ]:
from sklearn.metrics import accuracy_score 
accuracy = accuracy_score(y_true_flat, y_pred)

In [ ]:
accuracy

In [ ]:
image = Image.open('/kaggle/input/mepco-tropic-leaf/MepcoTropicLeaf-V1/Database/Ivy Gourd.zip/022_10.jpg').convert('RGB')
image

In [ ]:
transform1 = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to (224, 224)
    transforms.ToTensor(),           # Convert the image to a PyTorch tensor
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # Normalize the image with mean and standard deviation
        std=[0.229, 0.224, 0.225]
    )
])
normalized_image = transform1(image).unsqueeze(0)
print(normalized_image.shape)

In [ ]:
norm_image = normalized_image.cuda()
model.eval()
outputs = model(norm_image)
_, predicted = torch.max(outputs.data, 1)
print('Predicted class:', predicted.item())